In [17]:
import requests
import csv
import pandas 
import Tkinter
import tkFileDialog
import sys

   #buying       v-high, high, med, low
   #maint        v-high, high, med, low
   #doors        2, 3, 4, 5-more
   #persons      2, 4, more
   #lug_boot     small, med, big
   #safety       low, med, high
   #class        unacc, acc, good, vgood
    
s = raw_input('Use online csv file? y/n')

if s.lower() == 'y': 
    df = pandas.read_csv("%s%s%s" % ("https://raw.githubusercontent.com/ChristopheHunt/MSDA",
                 "---Coursework/master/Data%20602/Data%20602%20-",
                 "%20Advanced%20Programming/data/cars.data.csv"), header=None,  index_col=False)
elif s.lower() == 'n':
    root = Tkinter.Tk()
    filename = tkFileDialog.askopenfilename(parent=root) 
    df = pandas.read_csv(filename)
    root.withdraw()
    
else: print "input was not in a y/n format"


df.columns = [ 'buying', 'maint', 'doors','persons','lug_boot', 'safety', 'class']
df = df.apply(lambda df: df.astype('category')) 

safety_cat = ['low','med','high']
maint_cat = ['low','med','high','vhigh']
doors_cat = ['2','3','4','5more']

safety_issues = list(set(pandas.unique(df.safety.ravel())) - set(safety_cat))
if safety_issues != []:
    s = raw_input("Safety column contains unknown values: %s, this will be converted to NaN. Continue? y/n " % safety_issues)
    if s.lower() == 'n':
        sys.exit("End") 

#print all(value in pandas.unique(df.safety.ravel()) for value in safety_cat)
#print pandas.unique(df.safety.ravel())

Use online csv file? y/nn
Safety column contains unknown values: ['vhigh'], this will be converted to NaN values. Continue? y/n n


SystemExit: End

To exit: use 'exit', 'quit', or Ctrl-D.


In [14]:
df.safety.cat.set_categories(safety_cat, inplace = True)
df.maint.cat.set_categories(maint_cat, inplace = True)
df.doors.cat.set_categories(doors_cat, inplace = True)

In [15]:
#Print to the console the top 10 rows of the data sorted by 'safety' in descending order
results = df.sort_values(by = 'safety', ascending=[1])
results.head(10)

,buying,maint,doors,persons,lug_boot,safety,class
1387,vlow,NaN,NaN,1,vbig,NaN,NaN
722,med,vhigh,3,2,med,low,unacc
724,med,med,4,2,small,low,unacc
727,med,vhigh,3,more,big,low,unacc
729,vhigh,med,5more,2,big,low,unacc
732,high,med,3,2,big,low,unacc
735,med,vhigh,4,2,med,low,unacc
738,med,high,3,2,big,low,unacc
741,vhigh,low,4,4,small,low,unacc
720,low,vhigh,4,2,small,low,unacc


In [ ]:
#Print to the console the bottom 15 rows of the data sorted by 'maint' in ascending order
results = df.sort_values(by = 'maint', ascending = [0])
results.tail(15)

In [ ]:
#Print to the console all rows that are high or vhigh in fields 'buying', 'maint', and 'safety', 
# sorted by 'doors' in ascending order.  Find these matches using regular expressions.
results = df.sort_values(by = 'doors', ascending = [1])
mask = df[['buying', 'maint','safety']].apply(lambda column: column.str.contains('vhigh|high', regex=True)).any(axis=1)
results = df[mask].sort_values(by = 'doors', ascending = [1])
results.tail(10)


In [ ]:
#Save to a file all rows (in any order) that are: 'buying': vhigh, 'maint': med, 'doors': 4, and 
#'persons': 4 or more.  The file path can be a hard-coded location (name it output.txt) or use a dialog box.  
results = df.query("buying == 'vhigh' and maint == 'med' and doors == '4'")
results = results.query("persons == '4' or persons == 'more'")
results.to_csv("results.csv", index = False)
